In [1]:
from astropy.utils.data import conf
import numpy as np
conf.dataurl

'http://data.astropy.org/'

In [2]:
conf.remote_timeout

10.0

In [3]:
conf.remote_timeout = 10000

In [4]:
from astropy import units as u
from astropy.time import Time, TimeDelta
from astropy.coordinates import solar_system_ephemeris
solar_system_ephemeris.set("jpl")

from poliastro.bodies import Sun, Earth, Moon
from poliastro.ephem import Ephem

from poliastro.util import time_range
import matplotlib.pyplot as plt
import datetime
import random
from poliastro.twobody import Orbit
from poliastro.maneuver import Maneuver


EPOCH = Time("2020-10-06 12:05:50", scale="tdb")

start_date = datetime.date(2000, 1, 1)
end_date = datetime.date(2020, 1, 1)

time_between_dates = end_date - start_date
days_between_dates = time_between_dates.days
random_number_of_days = random.randrange(days_between_dates)
random_date = start_date + datetime.timedelta(days=random_number_of_days)

print(random_date)

2016-07-19


In [7]:
#TODO 
epochs = time_range(
    EPOCH - TimeDelta(2 * 31 * u.day), end=EPOCH + TimeDelta(2 * 31 * u.day), periods=100
)

st = EPOCH - TimeDelta(2 * 31 * u.day)
end = EPOCH + TimeDelta(2 * 31 * u.day)

date_launch = st
date_arrival = end

In [64]:
itokawa = Ephem.from_horizons("Itokawa", epochs, attractor=Earth)

earth = Ephem.from_body(Earth, epochs)
import math

ss_earth = Orbit.from_ephem(Sun, earth, date_launch)
ss_itokawa = Orbit.from_ephem(Sun, itokawa, date_arrival)

man_lambert = Maneuver.lambert(ss_earth, ss_itokawa)

ss_trans, ss_target = ss_earth.apply_maneuver(man_lambert, intermediate=True)
#print(ss_trans)
#print(ss_trans.classical())
#print(ss_trans.sample(max_anomaly=170 * u.deg))

anom = 130 * u.deg

x = np.array(itokawa.sample().x) * 1.495978707 * (10 ** 8)
y = np.array(itokawa.sample().y) * 1.495978707 * (10 ** 8)
z = np.array(itokawa.sample().z) * 1.495978707 * (10 ** 8)

sc_x = np.array(ss_trans.sample(max_anomaly=anom).x)
sc_y = np.array(ss_trans.sample(max_anomaly=anom).y)
sc_z = np.array(ss_trans.sample(max_anomaly=anom).z)

#print(x)
#print(sc_x)

filtered_pos_xyz = "filler"

for i in range(len(x)):
    m = 0.01*(i+1)
    current_pos = [sc_x[i],sc_y[i],sc_z[i]]
    distance = math.sqrt( ((x[i]-current_pos[0]) ** 2) + ((y[i]-current_pos[1]) ** 2) + ((z[i]-current_pos[2]) ** 2))
    #print(current_pos)
    #print(x[i])
    #print(y[i])
    #print(z[i])
    print(distance)
    if i == 0:
        pos_xyz = np.array([[x[i]-current_pos[0],y[i]-current_pos[1],z[i]-current_pos[2]]])
    else:
        pos_xyz = np.append(pos_xyz, [[x[i]-current_pos[0],y[i]-current_pos[1],z[i]-current_pos[2]]], axis = 0)
    if distance >= 10000 and distance <= 100000:
        if i == 0:
            filtered_pos_xyz = np.array([[x[i]-current_pos[0],y[i]-current_pos[1],z[i]-current_pos[2]]])
        else:
            filtered_pos_xyz = np.append(pos_xyz, [[x[i]-current_pos[0],y[i]-current_pos[1],z[i]-current_pos[2]]], axis = 0)

np.save('pos_xyz', pos_xyz)

if not filtered_pos_xyz == "filler":
    np.save('filtered_pos_xyz', filtered_pos_xyz)
#11078868.182499949

491223247.79378724
485795596.1953455
480378236.92360586
474972630.95753247
469580164.58467877
464202149.3963539
458839821.7359213
453494342.54029876
448166798.64995867
442858206.5136852
437569518.560074
432301631.04232
427055390.2901528
421831593.367649
416630980.7905448
411454222.74331707
406301903.3351353
401174507.42193216
396072412.1788367
390995883.2430623
385945074.2560387
380920028.71107703
375920683.4424381
370946873.3100243
365998336.6353989
361074720.83940315
356175587.77156854
351300418.4096779
346448616.9595947
341619514.753847
336812374.7417653
332026397.6564879
327260731.0110453
322514481.4226315
317786728.9802186
313076539.70635813
308382970.7389179
303705065.225983
299041839.55075306
294392269.55345875
289755281.50321317
285129749.5398663
280514498.1380821
275908307.2769904
271309918.6424634
266718042.07984206
262131362.12842473
257548544.65298894
252968243.54905438
248389107.35588852
243809785.58738464
239228934.69460967
234645223.87792334
230057341.35335293
225464002.

In [58]:
from poliastro.plotting import OrbitPlotter3D
plotter = OrbitPlotter3D()
#plotter.set_attractor()
plotter.plot(ss_trans)
#plotter.plot(ss_target)

plotter.plot_ephem(earth, date_launch, label="Earth at launch position")
plotter.plot_ephem(itokawa, date_arrival, label="Itokawa at arrival position")
plotter.plot_trajectory(
    ss_trans.sample(max_anomaly=130* u.deg), color="black", label="Transfer orbit"
)
plotter.set_view(30 * u.deg, 260 * u.deg, distance=3 * u.km)

In [8]:
import math

In [9]:
def R_2vect(vector_orig, vector_fin):
    R = np.zeros((3,3))
    # Convert the vectors to unit vectors.
    vector_orig = vector_orig / np.linalg.norm(vector_orig)
    vector_fin = vector_fin / np.linalg.norm(vector_fin)
    # The rotation axis (normalised).
    axis = np.cross(vector_orig, vector_fin)
    axis_len = np.linalg.norm(axis)
    if axis_len != 0.0:
        axis = axis / axis_len
    # Alias the axis coordinates.
    x = axis[0]
    y = axis[1]
    z = axis[2]
    # The rotation angle.
    angle = math.acos(np.dot(vector_orig, vector_fin))
    # Trig functions (only need to do this maths once!).
    ca = math.cos(angle)
    sa = math.sin(angle)
    # Calculate the rotation matrix elements.
    R[0,0] = 1.0 + (1.0 - ca)*(x**2 - 1.0)
    R[0,1] = -z*sa + (1.0 - ca)*x*y
    R[0,2] = y*sa + (1.0 - ca)*x*z
    R[1,0] = z*sa+(1.0 - ca)*x*y
    R[1,1] = 1.0 + (1.0 - ca)*(y**2 - 1.0)
    R[1,2] = -x*sa+(1.0 - ca)*y*z
    R[2,0] = -y*sa+(1.0 - ca)*x*z
    R[2,1] = x*sa+(1.0 - ca)*y*z
    R[2,2] = 1.0 + (1.0 - ca)*(z**2 - 1.0)
    return R
def rotationMatrixToEulerAngles(R) :
    #assert(isRotationMatrix(R))
    sy = math.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    if  not singular :
        x = math.atan2(R[2,1] , R[2,2])
        y = math.atan2(-R[2,0], sy)
        z = math.atan2(R[1,0], R[0,0])
    else :
        x = math.atan2(-R[1,2], R[1,1])
        y = math.atan2(-R[2,0], sy)
        z = 0
    return np.array([x, y, z])

def calculateEuler(path):
    vector_orig = [0,0,1]
    vec_orig_flip = -1*np.array(vector_orig)
    pos = np.load(path + '.npy')
    # pos_ is the position vector from the target body to the camera (pos_ = camera_position - target_body_position)
    for i in range(len(pos)):
        pos_ = -1*pos[i]
        R = R_2vect(vec_orig_flip, pos_) 
        angle0_blend = rotationMatrixToEulerAngles(R)
        if i == 0:
            euler_pos_xyz = np.array([angle0_blend])
        else:
            euler_pos_xyz = np.append(euler_pos_xyz, [angle0_blend], axis = 0)
    #print(euler_pos_xyz)
    np.save('euler_'+path, euler_pos_xyz)

calculateEuler('pos_xyz')
#print(filtered_pos_xyz)
if not filtered_pos_xyz == "filler":
    calculateEuler('filtered_pos_xyz')